In [1]:
#%matplotlib notebook
import os
import numpy as np
from landlab.io import read_esri_ascii, write_esri_ascii
from landlab import imshow_grid_at_node
from landlab.components import SpatialPrecipitationDistribution
from landlab.components import OverlandFlow
import matplotlib.pyplot as plt

from landlab import RasterModelGrid
from landlab.components import SoilInfiltrationGreenAmpt
from landlab import imshow_grid
from landlab import imshow_grid_at_node
from landlab.components import OverlandFlow
import numpy as np

from landlab.components.uniform_precip import PrecipitationDistribution
import numpy as np
from landlab import RasterModelGrid
import random
import matplotlib.pyplot as plt

In [2]:
# here we use an arbitrary, very small, "real" catchment
fname = 'hugo_site.asc'
mg, z = read_esri_ascii(fname, name='topographic__elevation')
mg.status_at_node[mg.nodes_at_right_edge] = mg.BC_NODE_IS_FIXED_VALUE
mg.status_at_node[np.isclose(z, -9999.)] = mg.BC_NODE_IS_CLOSED

hydraulic_conductivity = mg.ones('node')*1.e-6

plt.figure()
imshow_grid_at_node(mg, z, colorbar_label='Elevation (m)')
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'hugo_site.asc'

In [ ]:
plt.figure()
imshow_grid_at_node(mg, hydraulic_conductivity, colorbar_label='hc')
plt.show()

In [ ]:
imshow_grid(mg, mg.at_node['soil_water_infiltration__depth'])

In [ ]:
rain = SpatialPrecipitationDistribution(mg)
np.random.seed(26)  # arbitrary to get a cool-looking storm out every time

# get the storm simulator to provide a storm
# There's only one storm generated here in the time series, so easy enough to do.
# first, check the directory we need for saving exists, and make it if not:
if not os.path.exists('./rainfall'):
    os.makedirs('./rainfall')
for (storm_t, interstorm_t) in rain.yield_storms(style='monsoonal'):  # storm lengths in hrs
    mg.at_node['rainfall__flux'] *= 0.001  # because the rainfall comes out in mm/h
    mg.at_node['rainfall__flux'] *= 10.  # to make the storm heavier and more interesting!
    plt.figure()
    # plot up this storm
    imshow_grid_at_node(
        mg, 'rainfall__flux', cmap='gist_ncar', colorbar_label='Rainfall flux (m/h)'
    )
    plt.show()
    write_esri_ascii('./rainfall/rainfall.asc', mg, 'rainfall__flux', clobber=True)

In [ ]:
SI = SoilInfiltrationGreenAmpt(
    mg,hydraulic_conductivity=hydraulic_conductivity)
of = OverlandFlow(mg, steep_slopes=True)

In [ ]:
for filename in os.listdir('./rainfall'):  # for each file in the folder
    if filename.endswith(".asc"):  # ...that ends with .asc...
        # remove any rainfall field that already exists on the grid:
        try:
            _ = mg.at_node.pop('rainfall__flux')
        except KeyError:
            pass
        _, q_rain = read_esri_ascii(
            './rainfall/'+filename, grid=mg, name='rainfall__flux')
    else:
        continue
      
    mg.at_node['surface_water__depth'].fill(1.e-12)

    of = OverlandFlow(mg, steep_slopes=True)
    SI = SoilInfiltrationGreenAmpt(mg,hydraulic_conductivity=hydraulic_conductivity)
    node_of_max_q = 2126
    total_mins_to_plot = 60.  # plot 60 mins-worth of runoff
    plot_interval_mins = 10.  # show every 10 min
    min_tstep_val = 1.  # necessary to get the model going cleanly
    outlet_depth = []
    outlet_times = []
    mean_SID = []
    storm_elapsed_time = 0.
    total_elapsed_time = 0.
    last_storm_loop_tracker = 0.
    while total_elapsed_time < total_mins_to_plot * 60.:
        dt = of.calc_time_step()
        remaining_total_time = total_mins_to_plot * 60. - total_elapsed_time
        if storm_elapsed_time < storm_t * 3600.:
            remaining_storm_time = storm_t * 3600. - storm_elapsed_time
            dt = min((dt, remaining_total_time, remaining_storm_time, min_tstep_val))
        else:
            dt = min((dt, remaining_total_time, min_tstep_val))
        of.run_one_step(dt=dt)
        SI.run_one_step(dt=dt)
        total_elapsed_time += dt
        storm_elapsed_time += dt
        storm_loop_tracker = total_elapsed_time % (plot_interval_mins * 60.)
        # NB: Do NOT allow this plotting if there are multiple files in the folder
        if storm_loop_tracker < last_storm_loop_tracker:
            plt.figure()
            imshow_grid_at_node(
                mg,
                'soil_water_infiltration__depth',
                var_name='Stage (m)')
            plt.title('Soil_water_infiltraiton_depth t=' + str(total_elapsed_time//1) + 's')
            plt.show()
        last_storm_loop_tracker = storm_loop_tracker
        outlet_depth.append(mg.at_node['surface_water__depth'][node_of_max_q])
        mean_SID.append(np.mean(mg.at_node['soil_water_infiltration__depth']))
        outlet_times.append(total_elapsed_time)
        if storm_elapsed_time < storm_t * 3600.:
            mg.at_node['surface_water__depth'] += mg.at_node['rainfall__flux'] * dt / 3600.

In [ ]:
plt.figure()
plt.plot(outlet_times, outlet_depth, '-')
plt.xlabel('Time elapsed (s)')
plt.ylabel('Flood stage (m)')

In [ ]:
plt.figure()
plt.plot(outlet_times, mean_SID, '-')
plt.xlabel('Time elapsed (s)')
plt.ylabel('mean soil infiltration depth (m)')

In [ ]:
np_SID = np.array(mean_SID)
SID_diff = np.diff(np_SID)

In [ ]:
plt.figure()
plt.plot(outlet_times[0:-1], SID_diff, '-')
plt.xlabel('Time elapsed (s)')
plt.ylabel('mean soil infiltration depth difference over time')